# Inventories

Inventories are a fundamental doc entrypoint for our infrastructures. 

They contain a lot of informations, including:
    
    - ansible_user
    - configuration variables in [group_name:vars]
    - host grouping eg. by geographical zones in [group_name:children]
    
    
Files:


  - [inventory](/edit/notebooks/exercise-04/inventory)
    

In [2]:
cd exercise-04

/notebooks/exercise-04


In [3]:
!cat inventory

# I can group hosts in inventory

[web:children]
web_rome
web_milan

[web_rome:children]
web_rome_test
web_rome_prod

[web_rome_prod]
172.23.0.[3:4]

[web_milan]
172.24.0.[5:6]

# further host variables
[web_rome:vars]
ansible_username=root 


# Connect with docker 
[web_rome_test:vars]
ansible_connection=docker
ansible_docker_extra_args="-Htcp://172.17.0.1"

# The actual host reference
[web_rome_test]
ansible101_web_1

# 
# Don't need to be Ssh 
#  to be my local machine ;)
#
[course]
localhost    ansible_connection=local 

In [6]:
# The ping module is very useful. Use it whenever you want to check connectivity!
!ansible -m ping -i inventory all[1:3]


172.23.0.3 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.3 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.24.0.5 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.24.0.5 port 22: Connection timed out\r\n", 
    "unreachable": true
}
172.23.0.4 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.23.0.4 port 22: Connection timed out\r\n", 
    "unreachable": true
}


## Inventory scripts

In [22]:
#To create custom inventory scripts just use python ;) and set it in

!grep inventory  ansible.cfg # inventory = ./docker-inventory.py


inventory = inventory-docker.py


#### Exercise

in the official ansible documentation find at least 3 `ansible_connection=docker` parameters 

In [7]:
# List our containers. Note: this only works with docker-compose containers.
from __future__ import print_function
import docker
c=docker.Client(base_url="http://172.17.0.1:2375")
container_fmt = lambda x: (
    x['Names'][0][1:],
    x['Labels']['com.docker.compose.service'], 
    x['NetworkSettings']['Networks']['bridge']['IPAddress'],
)

for x in c.containers():
    print(*container_fmt(x), sep='\t\t')

ansible101_dev_1		dev		172.17.0.2
ansible101_ansible_1		ansible		172.17.0.5
ansible101_web_2		web		172.17.0.4
ansible101_web_1		web		172.17.0.3


In [34]:
# Ansible accepts
import json

inventories = {
    'web': {
        'hosts': ['ws-1', 'ws-2'],
    },
    'db': {
        'hosts': ['db-1', 'db-2'],
    }
}

# like this 
print(json.dumps(inventories, indent=1))
      

{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ]
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


In [35]:
# You can pass variables to generated inventories too
inventories['web']['host_vars'] = {
    'ansible_ssh_common_args': ' -o GSSApiAuthentication=no'
}

print(json.dumps(inventories, indent=1))


{
 "web": {
  "hosts": [
   "ws-1", 
   "ws-2"
  ], 
  "host_vars": {
   "ansible_ssh_common_args": " -o GSSApiAuthentication=no"
  }
 }, 
 "db": {
  "hosts": [
   "db-1", 
   "db-2"
  ]
 }
}


#### Exercise: 

Reuse the code in [inventory-docker.py](/edit/notebooks/exercise-04/inventory-docker.py) to print a json inventory that:

  - connects via docker to "web" hosts
  - connects via ssh to "ansible" hosts 

Test it in the cell below.


In [8]:
!ansible -m ping -i inventory-docker.py all --limit ansible


ERROR! Attempted to execute "inventory-docker.py" as inventory script: Inventory script (inventory-docker.py) had an execution error: DEBUG:docker.auth.auth:Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
DEBUG:docker.auth.auth:No config file found
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 172.17.0.1
DEBUG:urllib3.connectionpool:http://172.17.0.1:2375 "GET /v1.24/containers/json?all=0&limit=-1&trunc_cmd=0&size=0 HTTP/1.1" 200 None
DEBUG:root:Processing entry u'ansible101_dev_1\t\tdev\t\t172.17.0.2'
DEBUG:root:Processing entry u'ansible101_ansible_1\t\tansible\t\t172.17.0.5'
DEBUG:root:Processing entry u'ansible101_web_2\t\tweb\t\t172.17.0.4'
DEBUG:root:Processing entry u'ansible101_web_1\t\tweb\t\t172.17.0.3'
Traceback (most recent call last):
  File "/notebooks/exercise-04/inventory-docker.py", line 25, in <module>
    inventory['web']['ansible_ssh_common_args'] = ' -o StrictHostKeyChecking=no '
TypeError: list indices must be integers, not str
 


#### Exercise

Modify the [inventory-docker.py](/edit/notebooks/exercise-04/inventory-docker.py) to skip StrictHostKeyChecking only on web hosts.

In [9]:
# Test here your inventory

# Configurations

You may want to split inventory files and separate prod and test environment.

## Exercise: 

split inventory in two inventory files:

  - [prod](/edit/notebooks/exercise-04/prod)  for production servers 
  - [test](/edit/notebooks/exercise-04/test)  for test servers
  
Then use `ansible -i` to explicitly use the different ones.

In [23]:
# Use this cell to test the exercise

## group_vars and host_vars

You can move variables out of inventories - eg to simplify inventory scripts - and store them in files:

 - under `group_vars` for host groups
 - under `host_vars` for single hosts

In [12]:
!tree group_vars



group_vars
├── all
└── example

0 directories, 2 files


If you have different inventories, you can store different set of variable in custom files.
The `all` ones will be shared between all inventories

## Exercise: 
    
  - edit [group_vars/all](/edit/notebooks/exercise-04/group_vars/all) and move there all common variables from [inventory](/edit/notebooks/exercise-04/inventory) 

In [ ]:
# Test here the new inventory file

Inventory variables can store almost everything and even describe the architecture of your deployment


In [10]:
!cat group_vars/example

---
# This is an inventory variable example.

# Basic variables can be strings..
java_version: 1.8.0_131
http_proxy: "http://172.17.0.1:3128"           # ...strings with quotes

# .. numbers
timestamp: 20170501
default_int: 0

# boolean (see http://yaml.org/type/bool.html)
enable_selinux: "yes"
enable_feature: "True"

#
# But you can have
#

a_list:
  - curl
  - wget

a_map:
  name: /etc/hosts
  mode: 0644

a_list_of_map:
  - name: /etc/hosts
    mode: 0644
  - name: /etc/shadow
    mode: 0000


We can even mix and mojo `group_vars` and `inventory`, as we'll see in the next lessons.





### host_vars

Host vars can be used in automated or cloud deployments where:

  - every new host or vm, at boot, populate its own entries in `host_vars` (Eg. via file)
  - ansible is run after that setup and uses `host_vars` to configure the server and expose that values to the other machines.